In [1]:
import datetime
import logging
import pickle

import pandas as pd

from data_collection.models import Game, Round
from data_collection.questions_candidates import get_question_candidates
from data_collection.questions_pick import pick_question
from data_collection.semaphore import set_semaphore
from data_collection.guess import guess
from data_collection.answer import answer
from data_collection.simulate_game import run_games


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from dotenv import load_dotenv
from openai import AsyncOpenAI

from data_collection.utils import call_gpt4, dataset

load_dotenv()

True

In [2]:
set_semaphore("gpt-4", 50)

In [3]:
client = AsyncOpenAI()

In [4]:
# samples_by_category = {
#     "things": 100,
#     "city": 40,
#     "country": 20,
#     "landmark": 40,
# }
#OVERALL

In [5]:
# samples_by_category = {
#     "things": 25,
#     "city": 10,
#     "country": 5,
#     "landmark": 10,
# }
# 50 examples

In [6]:
samples_by_category = {
    "things": 50,
    "city": 20,
    "country": 10,
    "landmark": 20,
}
# 100 examples

In [7]:
keywords_by_category = pickle.load(open("../keywords_by_category.pkl", "rb"))

In [8]:
keywords_by_category.keys()

dict_keys(['city', 'country', 'landmark', 'things'])

In [9]:
for key in keywords_by_category:
    keywords_by_category[key] = keywords_by_category[key].sample(samples_by_category[key])
all_dfs = pd.concat(keywords_by_category.values())
keywords_flat = list(all_dfs.keyword)

In [10]:
keywords_flat

['glasgow',
 'nairobi',
 'riyadh',
 'ulaanbaatar',
 'fresno',
 'porto',
 'singapore',
 'ingolstadt',
 'milan',
 'essen',
 'dunfermline',
 'fukuoka',
 'athens',
 'nurumberg',
 'juarez',
 'osaka',
 'athens',
 'odense',
 'jacksonville',
 'fortaleza',
 'south africa',
 'congo',
 'latvia',
 'belarus',
 'albania',
 'norway',
 'mozambique',
 'malta',
 'denmark',
 'madagascar',
 'mount whitney',
 'rhine',
 'yangtze river',
 'yenisei river',
 'nanga parbat',
 'amazon',
 'aconcagua',
 'ganges',
 'mount shasta',
 'pikes peak',
 'aoraki',
 'iztaccihuatl',
 'makalu',
 'mississippi river',
 'haleakala',
 'annapurna',
 'grand teton',
 'kangchenjunga',
 'kilimanjaro',
 'fitz roy',
 'water dish',
 'oak table',
 'seagrass bed',
 'wall street',
 'aspirin',
 'bumble bee',
 'incense',
 'vegetarian chili',
 'fire alarm',
 'air compressor',
 'swing set',
 'throw blanket',
 'dental chair',
 'clog',
 'cardboard',
 'photocopier',
 'manicure set',
 'television',
 'wonton',
 'silicone',
 'wall mirror',
 'tweezers

In [11]:
games = await run_games(client, keywords_flat, include_answer=True)
# games = await run_games(client, ["ice tea"], include_answer=True)

Game glasgow, round 0
Game nairobi, round 0
Game riyadh, round 0
Game ulaanbaatar, round 0
Game fresno, round 0
Game porto, round 0
Game singapore, round 0
Game ingolstadt, round 0
Game milan, round 0
Game essen, round 0
Game dunfermline, round 0
Game fukuoka, round 0
Game athens, round 0
Game nurumberg, round 0
Game juarez, round 0
Game osaka, round 0
Game athens, round 0
Game odense, round 0
Game jacksonville, round 0
Game fortaleza, round 0
Game south africa, round 0
Game congo, round 0
Game latvia, round 0
Game belarus, round 0
Game albania, round 0
Game norway, round 0
Game mozambique, round 0
Game malta, round 0
Game denmark, round 0
Game madagascar, round 0
Game mount whitney, round 0
Game rhine, round 0
Game yangtze river, round 0
Game yenisei river, round 0
Game nanga parbat, round 0
Game amazon, round 0
Game aconcagua, round 0
Game ganges, round 0
Game mount shasta, round 0
Game pikes peak, round 0
Game aoraki, round 0
Game iztaccihuatl, round 0
Game makalu, round 0
Game miss

2024-07-08 20:35:11,636 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,667 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,673 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,682 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,710 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,719 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,752 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,759 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:11,789 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

Game norway, round 1


2024-07-08 20:35:21,985 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:21,993 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:22,002 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:22,039 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:22,045 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:22,075 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:22,106 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:35:22,118 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Game haleakala, round 1
Game kilimanjaro, round 1
Game albania, round 1
Game fresno, round 1
Game nanga parbat, round 1


...

2024-07-08 20:42:45,077 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:45,383 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:46,010 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:46,410 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:46,863 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:47,215 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:47,625 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:47,773 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:48,787 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

Game clog, round 19


2024-07-08 20:42:51,049 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:51,341 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:52,073 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:53,217 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Game bacon, round 19


2024-07-08 20:42:53,539 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:53,695 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:53,929 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:54,888 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:55,275 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:57,635 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:42:57,731 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:43:00,422 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-08 20:43:03,719 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [24]:
games[96]

Game(keyword='earthworm', rounds=[Round(question='Is it a living thing?', answer='yes', guess='is it an animal'), Round(question='Is it an animal?', answer='yes', guess='is it a mammal'), Round(question='Is it a mammal?', answer='no', guess='bird'), Round(question='Is it a bird?', answer='no', guess='reptile'), Round(question='Does it have a backbone?', answer='no', guess='insect'), Round(question='Does it have more than four legs?', answer='no', guess='worm'), Round(question='Is it an insect?', answer='no', guess='snail'), Round(question='Does it live in water?', answer='no', guess='spider'), Round(question='Is it a type of worm?', answer='yes', guess='earthworm')], win=True)

In [13]:
dataset[1000]

{'prompt': 'You are playing "20 Questions" game.\n    \nHere are the previous questions and answers:\n    Question: Is it a thing? Answer: yes Your guess: is it an electronic device\nQuestion: Is it man-made? Answer: no Your guess: tree\n\n\nThe guessed keyword is "annapurna".\n\nAnother player is asking: "Is it a living organism?".\n\nYou goal is to answer this question about the keyword as truthfully as possible.\nYou can only answer YES or NO. Nothing else. Even if you don\'t know, even if there\'s no simple yes/no answer - you still HAVE TO answer YES or NO.\nGiven this harsh constraint, still do your best to be useful, truthful, and helpful to the other player.\n\nFormat your answer like this: First, reason about your answer for 1-3 sentences. If the answer is obvious - state it. If not - reason about whether YES or NO would be more helpful and truthful.\nThen, your last line should be the answer, only YES or NO.\nDon\'t output anything else.\n',
 'response': 'Annapurna is a mount

In [17]:
len(dataset)

6016

In [16]:
dt = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
pickle.dump(dataset, open(f"dataset_{dt}.pkl", "wb"))

In [15]:
dataset[0]

{'prompt': 'You are playing "20 Questions" game.\n    \n\n    \nYou need to generate 10 candidates for the next question to ask.\nYou should mix safe questions with more creative and non-orthodox ones.\nSafe questions are usually ones that split the space of possibilities in half.\nIf you have a few questions left, you can start acting more aggressively, intuitively checking some less probable, but high-reward hypotheses.\nConsider asking non-standard questions like "Does the keyword consist of a single word?, however only do it if you don\'t know it yet, and if you think that it\'s important to know at this point in the game.\nConsider asking questions like "Does the keyword start with letter \'A\'?" or "Does the keyword contain letter \'E\'?". However, don\'t overdo it, focus on more substantial questions.\n\nFormat your answer like this: output every question on a new line. Don\'t output anything else.\n',
 'response': 'Is it a place?\nIs it a thing?\nIs it a concept?\nIs it somethi

In [19]:
len([game for game in games if game.win])

22

In [18]:
games[1]

Game(keyword='jalapa', rounds=[Round(question='Is it a place?', answer='yes', guess='is it a country'), Round(question='Is it a country?', answer='no', guess='eiffel tower'), Round(question='Is it a city?', answer='yes', guess='paris'), Round(question='Is it located in Europe?', answer='no', guess='new york city'), Round(question='Is it located in North America?', answer='yes', guess='los angeles'), Round(question='Is it located in the United States?', answer='no', guess='toronto'), Round(question='Is it located in Canada?', answer='no', guess='mexico city'), Round(question='Is it located in Mexico?', answer='yes', guess='guadalajara'), Round(question='Is it a coastal city?', answer='no', guess='monterrey'), Round(question='Is it located in the central part of Mexico?', answer='no', guess='tijuana'), Round(question='Is it located in the northern part of Mexico?', answer='no', guess='oaxaca'), Round(question='Is it located in the southern part of Mexico?', answer='no', guess='puebla'), 

In [1]:
# game = await run_game(client, "headphones")